In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import auc, plot_roc_curve, average_precision_score
from sklearn.linear_model import LogisticRegression


from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import classification_report, confusion_matrix
from pprint import pprint
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
from sklearn.utils import shuffle

df = pd.read_csv("Data.csv", sep=";")
y = df["Target"]
X = df.iloc[:,1:-1]

# Standardize data
scaler.fit(X)
X = scaler.transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, test_size=0.30, random_state=1, shuffle=True)
X_train_val, X_test_val, y_train_val, y_test_val = train_test_split(X_train, 
                                                                      y_train, test_size=0.2, random_state=1, shuffle=True)

## Training data

In [ ]:
LR = LogisticRegression(class_weight="balanced",
                        max_iter=3000,
                        solver="newton-cg",
                        penalty="l2",
                        C=0.01,
                        warm_start=False, 
                        n_jobs=-1, 
                        verbose=2,
                        random_state = 1
                       )

LR.fit(X_train_val, y_train_val)
pred = LR.predict(X_test_val)
print(classification_report(y_test_val,pred,digits=3))
print(confusion_matrix(y_test_val,pred))
print(f"Average precision score {average_precision_score(y_test_val, pred):.3f}")

## Test data

In [ ]:
LR = LogisticRegression(class_weight="balanced",
                        max_iter=3000,
                        solver="newton-cg",
                        penalty="l2",
                        C=0.01,
                        warm_start=False, 
                        n_jobs=-1, 
                        verbose=2,
                        random_state = 1
                       )
LR.fit(X_train, y_train)
pred = LR.predict(X_test)
Accur.append(average_precision_score(y_test, pred))
print(classification_report(y_test,pred,digits=3))
print(confusion_matrix(y_test,pred))
print(f"Average precision score {average_precision_score(y_test_val, pred):.3f}")

## Cross validation

In [ ]:
# 0.882508 using {'C': 0.1, 'penalty': 'l2', 'solver': 'newton-cg'}

LR = LogisticRegression(class_weight="balanced",max_iter=3000, 
                        C=0.01, penalty="l2", solver="newton-cg", warm_start=False, 
                        verbose=2,random_state = 1)
cv = StratifiedKFold(n_splits=5)


tprs = []
aucs = []
mean_fpr = np.linspace(0, 1, 100)

fig, ax = plt.subplots()
for i, (train, test) in enumerate(cv.split(X_train, y_train)):
    LR.fit(X_train[train], y_train.iloc[train])
    viz = plot_roc_curve(LR, X_train[test], y_train.iloc[test],
                         name='ROC fold {}'.format(i),
                         alpha=0.3, lw=1, ax=ax)
    interp_tpr = np.interp(mean_fpr, viz.fpr, viz.tpr)
    interp_tpr[0] = 0.0
    tprs.append(interp_tpr)
    aucs.append(viz.roc_auc)

ax.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r',
        label='Chance', alpha=.8)

mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)
ax.plot(mean_fpr, mean_tpr, color='b',
        label=r'Mean ROC (AUC = %0.3f $\pm$ %0.3f)' % (mean_auc, std_auc),
        lw=2, alpha=.8)

std_tpr = np.std(tprs, axis=0)
tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
ax.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2,
                label=r'$\pm$ 1 std. dev.')

ax.set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05],
       title="d) Receiver operating characteristics: LRC")
ax.legend(loc="lower right")
plt.savefig("LR.pdf", dpi=600, transparent=True)
plt.savefig("LR.jpg", dpi=600, transparent=True)
plt.show()

## Y Randomisierung

In [ ]:
from sklearn.metrics import average_precision_score
Accur = []
for i in range(1,100):
    y_rand = shuffle(y)
    X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y_rand, test_size=0.30, random_state=1)
    LR.fit(X_train, y_train)
    pred = LR.predict(X_test)
    Accur.append(average_precision_score(y_test, pred))
    print(classification_report(y_test,pred,digits=3))
    print(confusion_matrix(y_test,pred))
print(f"Mean f1 score = {np.mean(Accur):.3f}, std {np.std(Accur):.3f}")